# Diskussion & grafische Darstellung der Algorithmen
-------
Hinweis:
Die Daten zur Auswertung befinden sich im Dokument "Auswertung_Daten_Roundtrip.xslx".

# Auswahl der Algorithmen

Für die Auswertung der Algorithmen wurden die folgenden Benchmarks gewählt:

* Trap
* Bottleneck
* Fat Bottleneck
* B Rush
* Circle of Death

Aufgrund dieser Auswahl wird sichergestellt, dass die Algorithmen vor möglichst viele unterschiedliche Herausforderungen gestellt werden. Die Algorithmen werden in „Fallen“ gelockt, aus denen sie wieder herausfinden müssen. Es werden sowohl kurze als auch lange schmale Durchführungen überprüft. Außerdem befinden sich in den Umgebungen zum Teil mehrere kleine oder größere Hindernisse, sowie runde und labyrinthartige Formen.

# Implementierung des Roundtrip-Path Planers

## BasicPRM

Zugehörige Python-Klasse: [IPBasicPRM_Roundtrip](../02_eigeneNotebooks/BasicPRM/IPBasicPRM_Roundtrip.py)

<img src="./Bilder/AblaufBasicPRM.png" alt="Ablaufdiagramm BasicPRM" width="900"/>

Im Flussdiagramm rechts in der Abbildung werden der Ablauf und die Eventualitäten dargestellt. Die Vorbedingungen sind hierbei, dass die Roadmap vorhanden ist und die Start und Zwischenziele im Graphen sind. Letztere werden in der IPTestSuite definiert.
Der Ablauf des BasicPRM kann wie folgt beschrieben werden:
  
<ol>
    <li>Bestimme erstes Zwischenziel anhand euklidischem Abstand</li>
    <li>Erstelle Versuchspfad zu erstem Zwischenziel</li>
    Solange nicht alle Ziele erreicht sind:<br>
        <ul>
            Für jeden Schritt im Versuchspfad:<br>
            1. Füge aktuellen Schritt dem Lösungspfad an <br>
            2. Bestimme nächstes Zwischenziel mit euklidischem Abstand
            <li>Prüfe:
                <ol>
                    <li>Nächstes Zwischenziel hat sich geändert --> Umplanung, erstelle neuen Versuchspfad zu neuem nächsten Zwischenziel</li>
                    <li>Zwischenziel erreicht --> Erstelle Versuchspfad zum nächsten Zwischenziel, setze Schleife zurück</li>
                    <li>Zwischenziel hat sich nicht geändert --> Führe nächsten Schritt im Versuchspfad weiter aus</li>
                    <li>Sind alle Ziele erreicht --> Beende Algorithmus</li>
                </ol>
            </li>
        </ul>
    </li>
</ol>

## VisibilityPRM 


Zugehörige Python-Klasse: [IPVisibilityPRM_Roundtrip](../02_eigeneNotebooks/VisibilityPRM/IPVisibilityPRM_Roundtrip.py)

<img src="./Bilder/AblaufVisibility.png" alt="Ablaufdiagramm VisibilityPRM" width="900"/>

Im Flussdiagramm rechts in der Abbildung werden der Ablauf und Eventualitäten dargestellt. Der Ablauf des VisibilityPRMs unterscheidet sich nur geringfügig zum BasicPRM. Im Gegensatz zum BasicPRM werden beim VisibilityPRM zu Beginn Nodes, welche sich sehen können, direkt miteinander verbunden. Der verbleibende Ablauf des Algorithmus gleicht dann dem Ablauf des BasicPRMs.

## Optimierter VisibilityPRM

Zugehörige Python-Klasse: [CC_IPVisibilityPRM_Roundtrip](../02_eigeneNotebooks/VisibilityPRM/CC_IPVisibilityPRM_Roundtrip.py)

In Aufgabenstellung 1c) dieser Projektarbeit soll eine bestimmte Eigenschaft des VisibilityPRMs verändert werden, um diesen zu optimieren. Als Ergebnis einer Analyse der LearnRoadmap-Funktion wurde erkannt, dass die Gesamtanzahl der generierten Nodes verringert werden kann. Der Algorithmus erzeugt immer 300 Nodes, wobei die Anzahl der Nodes durch eine Konfiguration festgelegt wird. Wenn die Anzahl der Nodes sinnvoll reduziert werden kann, ist es möglich durch die Reduktion der Funktionsausrufe Rechenzeit zu sparen und somit den Algorithmus effizienter zu gestalten. Jedoch muss darauf geachtet werden, dass ausreichend viele Nodes erzeugt werden, da der Algorithmus bei einer zu geringen Nodeanzahl keinen Lösungspfad findet.

Im Folgenden wird eine optimierte Version des VisibilityPRMs beschrieben.

<img src="./Bilder/Änderungen_Visibility.png" alt="Optimierte Version VisibilityPRM" width="900"/>

Im Flussdiagramm rechts in der Abbildung werden der Ablauf und Eventualitäten dargestellt. Die Optimierung findet in der LearnRoadmap-Methode der CC_VisibilityPRM-Klasse statt. CC im Klassennamen steht für ConnectedComponents. Der Unterschied zum Standard-Visibility ist, dass initiale Nodes nur solange erzeugt werden, bis ein Pfad zwischen allen Zwischenzielen und dem Start vorhanden ist. Durch Verwendung einer Schleife wird dieses Kriterium vor jeder einzelnen Nodegenerierung überprüft.

Das Resultat der Optimierung bedeutet:

* Durchschnittlich circa 96 % weniger Nodes über alle Benchmarks
* Geringerer Ausführungszeit im Vergleich zum Standard-Visibilty (3s bei Standard, 0.5s bei optimierter Version)

## LazyPRM

Zugehörige Python-Klasse: [IPLazyPRM_Roundtrip](../02_eigeneNotebooks/LazyPRM/IPLazyPRM_Roundtrip.py)

<img src="./Bilder/Ablauf_lazy.png" alt="Ablaufdiagramm LazyPRM" width="900"/>

Im Flussdiagramm rechts im Bild werden der Ablauf und Eventualitäten dargestellt. Wie auch bei vorherigen Algorithmen, wird das erste Zwischenziel mittels des euklischem Abstand gefunden. Danach wird ein Versuchspfad zum ersten Zwischenziel ermittelt.
Anschließend wird geprüft, ob es eine Kollison zwischen dem aktuellen Schritt im Versuchspfad und letzten Pfadelement auftritt. 

Im Falle einer Kollision, werden neue Nodes hinzugefügt, um die Wahrscheinlichkeit eines kollisionsfreien Pfads zu erhöhen. Ebenfalls wird ein Abbruchzähler erhöht. Danach wird ein neuer Versuchspfad erstellt und die Schleife zurückgesetzt. Erreicht der Abbruchzähler sein Maximum wird der Algorithmus beendet und somit kein Pfad gefunden. Die Parameter der neuen Nodes und des Abbruchzählers werden in der Konfiguration festgelegt.

Im Falle keiner Kollision, wird die Distanz zum nächsten Zwischenziel ermittelt. Der verbleibende Ablauf unterscheidet sich dann nicht vom Basic oder VisibilityPRM.
Nach Erreichen des Endziels wird der Lösungspfad optimiert. Diese Vorgehensweise wird in [Aufgabe 2](Aufgabe2_Optimierung_und_Glaettung.ipynb) näher erläutert.

# Auswertung des Roundtrip-Path Planers

## Durchführungszeit

Nachfolgend dargestellt sind die durchschnittlichen Durchführungszeiten der zuvor beschriebenen Algorithmen anhand von 25 Wiederholungen pro Benchmark. Auf der X-Achse befinden sich die Benchmarks, auf der Y-Achse sind die Durchführungszeiten in Sekunden aufgeführt.

<img src="./Bilder/Folie9.png" alt="Diagramm Auswertung Durchführungszeit" width="900"/>

Der schnellste Algorithmus ist der LazyPRM, gefolgt vom optimierten VisibilityPRM. Anschließend folgt der BasicPRM, der Standard-VisibilityPRM stellt den langsamsten Algorithmus dar.

Der LazyPRM ist am schnellsten, da weniger rechenzeitintensive Kollisionschecks durchgeführt werden. 
Der optimierte VisbilityPRM (CC_Visibility) hat eine stark reduzierte Anzahl an Nodes, damit ist dieser deutlich schneller als der Standard-VisibilityPRM.
Der BasicPRM befindet sich im Mittelfeld, da vergleichsweise viele Kollisionschecks ausgeführt werden.

## Erfolgsrate

In der nachfolgenden Grafik wird die Erfolgsrate aufgeführt. Dies bedeutet die Wahrscheinlichkeit, dass ein Algorithmus einen Lösungspfad findet. Auf der X-Achse befinden sich die Benchmarks, auf der Y-Achse befindet sich die Erfolgsrate in Prozent. Dieser Auswertung liegen Daten aus 75 Wiederholungen je Benchmark zugrunde.

<img src="./Bilder/Folie10.png" alt="Diagramm Auswertung Erfolgsrate" width="900"/>

Es fällt auf, dass der Standard- sowie der optimierte VisibilityPRM nahezu eine maximale Erfolgsrate haben. 
Hierbei wird deutlich, dass mit einer höheren Nodeanzahl nicht automatisch eine höhere Erfolgsrate des VisibilityPRM einhergeht.

Die Erfolgsraten des BasicPRMs und des LazyPRMs befinden sich im Mittelfeld.
Auffällig ist, dass besonders schmale Gänge der Benchmarks "Fat Bottleneck" und "Circle Of Death" den Algorithmen "BasicPRM" und "LazyPRM" Probleme bereiten.
Speziell beim LazyPRM ist zu erkennen, dass der Algorithmus viele Schritte benötigt, um größere Distanzen zu überwinden und um aus "Fallen" in Benchmarks herauszufinden. In diesen Fällen steigt der Zähler des Abbruchkriteriums, was erheblichen negativen Einfluss auf die Erfolgsrate hat.

## Größe der Roadmap

In der nächsten Grafik wird die Größe der Roadmap, dies bedeutet die Anzahl der erzeugten Nodes in der Umgebung, dargestellt. Auf der X-Achse befinden sich die Benchmarks, auf der Y-Achse befindet sich die Größe der Roadmap. Dieser Auswertung liegen wie zuvor Daten aus 75 Wiederholungen je Benchmark zugrunde.

<img src="./Bilder/Folie11.png" alt="Diagramm Auswertung Roadmap" width="900"/>

Hierbei ist anzumerken, dass der optimierte VisibilityPRM die mit Abstand kleinste Roadmap von durchschnittlich circa 12 Nodes besitzt. Der Standard-VisibilityPRM hat eine einheitliche Roadmap Größe von 300 Nodes. Prozentual gesehen werden beim optimierten VisibilityPRM, wie bereits erwähnt, im Schnitt circa 96 % weniger Nodes erzeugt. Das BasicPRM Verfahren erzeugt eine feste Roadmap-Größe von 206 Nodes. Der LazyPRM Algorithmus erzeugt durchschnittlich circa 273 Nodes. Die größte Roadmap, mit durchschnittlich 470 Nodes, erzeugt ebenfalls der LazyPRM beim Benchmark "Circle of Death". Dies ist auf die labyrinthartigen, engen Gänge des Benchmarks zurückzuführen, da hier besonders viele Kollisionen auftreten.

## Länge des Pfades

In der folgenden Grafik wird eine Auswertung der durchschnittlichen Pfadlänge dargestellt. Auf der X-Achse sind die Benchmarks aufgetragen, auf der Y-Achse die Pfadlänge in Schritten. Grundlage dieser Auswertung stellen 13 Wiederholungen dar.

<img src="./Bilder/Pfadlaenge.png" alt="Diagramm Auswertung Pfadlänge" width="900"/>

Der BasicPRM hat, im Vergleich zu den anderen Algorithmen, die durchschnittlich größste Pfadlänge von circa 51 Nodes. Darauf folgt der LazyPRM, welcher eine Anzahl von durchschnittlich circa 17 Pfadelementen besitzt. Der Standard- und optimierte VisibilityPRM haben die geringste Anzahl von durchschnittlich circa 11 Pfadelementen. 

Innerhalb des VisibilityPRMs hat die Verringerung der Nodes, keine Auswirkung auf die Länge des Pfades. Dies unterstreicht nochmals, dass die zusätzlichen Nodes für die Pfadfindung des VisibilityPRMs nicht relevant sind. 

Da beim LazyPRM Duplikate im Lösungspfad enthalten sein können, werden diese während der Ertellung des Lösungspfades entfernt. Das Vorgehen ist hierbei in dem Notebook "Aufgabe2_Optimierung_und_Glaettung" [Aufgabe2](Aufgabe2_Optimierung_und_Glaettung.ipynb) beschrieben. Bei den anderen Algorithmen tritt dieses Verhalten nicht auf.

Besonders auffällig ist, dass der Benchmark "B_Rush" und "Circle of Death" benchmarkübergreifend die längsten Pfadlängen aufzeigen. Dies ist auf viele Verwinkelungen und enge Pfade zurückzuführen. Die Algorithmen können in diesen Benchmarks zum Teil immer nur kurze Schritte machen, weshalb mehr Nodes benötigt werden, um in der Benchmarkumgebung zu navigieren.

Benchmarks ohne Lösungspfad wurden nicht mit in dieser Auswertung berücksichtigt.


# Herausforderungen eines Roundtrip Path Planners

**Anfälligkeit für Endlosschleifen:**

Aufgund der iterativen Überprüfung und Änderung des Zwischenziels, kommt es bei einer ungünstigen Konstellation von Nodes zu Endlosschleifen während der Pfadfindung. Gelöst wurde dies durch die Erkennung des Schleifenmusters und der anschließenden festen Entscheidung für ein Zwischenziel.

**Duplikate in Lösungspfad:**

Aufgrund der iterativen Aktualisierung des Versuchspfads, bei dem die Verwendung von bereits genutzten Nodes erlaubt sein muss, kann es zu Duplikaten im Lösungspfad kommen. Diese Herausforderung wurde durch die Optimierung der Teilabschnitte des Lösungspfads bewältigt, indem Duplikate entfernt wurden (siehe [Aufgabe 2](Aufgabe2_Optimierung_und_Glaettung.ipynb)).